In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import requests
import datetime
import time
import pytz
from concurrent.futures import ThreadPoolExecutor
import numpy as np

In [2]:
# Inicialização da variavel para que na consulta request não gere problemas
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

In [3]:
# inicialização das listas que receberão os valores
dividendos_valores = []
nomes_fii = []
valores_fii = []
data_hora_atual = []
links = []

In [4]:
# Leitura da lista de fundos que serão consultados
df2 = pd.read_csv(
    "fundosListados.csv",
    engine="python",
    encoding="latin-1",
    header=None,
    skiprows=1,
    skip_blank_lines=True,
)

In [5]:
# Definição do nome de pesquisa e do link de pesquisa dos fundos
nome = df2.iloc[0 : len(df2), 3].values
link = "https://www.fundsexplorer.com.br/funds"

Código em questão faz consulta sem divisão de trabalho.
  É funcional porém tende a demorar mais para consulta completa

In [6]:

# def consulta():
#   start = time.time()
#   for i in range (len(df2)):
#     link_name = f'{link}/{nome[i]}11'
#     # print(f'Start - {i} - {nome[i]}')
#     dividendos = requests.get(link_name, headers=headers).text

#     teste_dividendos = BeautifulSoup(dividendos, 'lxml')
#     teste_if = teste_dividendos.find("div",class_= "headerTicker__content__price").find('p')
#     if teste_if:
#       value_fii = teste_if.text
#     teste2 = teste_dividendos.find('div', class_="indicators historic").find('b')
#     if teste2:
#         div = teste2.text
#     else:
#       div = ''
#     dividendos_valores.append(div)
#     nomes_fii.append(nome[i])
#     valores_fii.append(value_fii)
#     links.append(link_name)
#     # print(f'OK - {i} - {nome[i]}')
#   df = pd.DataFrame(data={'Nome Fundo':nomes_fii,'Valor Fundo':valores_fii,'Valor dividendos':dividendos_valores,'Data':data_atual,'Links':links})
#   df.to_csv('drive/MyDrive/Fundos Imobiliarios/Fundos_imobiliarios_novo.csv',mode='a', header=False)
#   end = time.time()
#   print(end - start)

In [7]:
# Consulta individual foi gerada para acelerar procedimento
# Basicamente uma função que recebe o indice do nome e retorna a consulta indidual deles
def consulta_individual(i):
    link_name = f"{link}/{nome[i]}11"
    dividendos = requests.get(link_name, headers=headers).text
    teste_dividendos = BeautifulSoup(dividendos, "lxml")
    if teste_dividendos.find("div", class_="headerTicker__content__price").find("p").text and teste_dividendos.find("div", class_="headerTicker__content__price").find("p").text == 'R$ 0,00':
      return
    else:
      consulta_teste = teste_dividendos.find("div", class_="headerTicker__content__price").find("p")
      # print(consulta_teste)
    if consulta_teste != 'N/A':
        value_fii = consulta_teste.text
    else:
      return nome[i], "0.00", "0.00", link_name
      # print(value_fii)
    if teste_dividendos.find("div", class_="indicators historic").find("b").text == 'N/A' or teste_dividendos.find("div", class_="indicators historic").find("b").text == 'NaN':
        div = "0.00"
    else:
        div = teste_dividendos.find("div", class_="indicators historic").find("b").text
        # print(teste_dividendos.find("div", class_="indicators historic").find("b").text)

    # print(f"OK - {nome[i]}")
    # print(nome[i], value_fii, div, link_name)
    return nome[i], value_fii, div, link_name


In [8]:
# Gerencia processo de consulta e divide em subconsultas através do ThreadPoolExecutor
# Aceleração do processo irá depender de quantos núcleos processador disponibiliza para processo
def consulta_geral():
  start = time.time()
  # Indicar entre 1 - 8 no parametro max_workers
  with ThreadPoolExecutor(max_workers=8) as executor:

    resultados = executor.map(consulta_individual, range(len(df2)))
  for resultado in resultados:
    if(resultado):
      nomes_fii.append(resultado[0])
      valores_fii.append(resultado[1])
      dividendos_valores.append(resultado[2])
      links.append(resultado[3])
    else:
      continue

  data_columns = ["Nome_Fundo", "Valor_Fundo", "Valor_dividendos", "Data", "Links"]
  data_teste = datetime.datetime.now(pytz.timezone("America/Sao_Paulo"))
  data_atual = data_teste.strftime("%d-%m-%y  %H:%M:%S")
  df = pd.DataFrame(data={'Nome_Fundo':nomes_fii,'Valor_Fundo':valores_fii,'Valor_dividendos':dividendos_valores,'Data':data_atual,'Links':links}, columns = data_columns)


  df['Valor_Fundo'] = df['Valor_Fundo'].str.replace(r'[R$ ]','',regex=True)
  df['Valor_Fundo'] = df['Valor_Fundo'].apply(lambda x: float(x.replace(".","").replace(",",".")))
  df['Valor_dividendos'] = df['Valor_dividendos'].apply(lambda x: float(x.replace(".","").replace(",",".")))
  df = df.astype({
    'Nome_Fundo': str,
    'Valor_Fundo': np.float64,
    'Valor_dividendos': np.float64,
    'Data': 'datetime64[ns]',
    'Links': str,
})
  df['Porcentagem'] =  (df['Valor_dividendos'] / df['Valor_Fundo']) * 100
  df.to_csv(
    "Fundos_imobiliarios_novo.csv",
    mode="a",
    header=None,
    index= False
  )
  end = time.time()
  print(end - start)

In [9]:
# Chamada da consulta para teste
consulta_geral()

58.28834867477417


In [10]:
# Função para consulta n quantidade de tempo
# if __name__ == "__main__":
#     while True:
#         consulta_geral()
#         time_wait = 10
#         print(f"Aguarde {time_wait} minutos")
#         time.sleep(time_wait * 60)

In [11]:

dfConsulta = pd.read_csv(
    "Fundos_imobiliarios_novo.csv",
    encoding="latin-1",
    header = None
)
dfConsulta.rename(columns={
    0:'Nome_Fundo',
    1:'Valor_Fundo',
    2:'Valor_dividendos',
    3:'Data',
    4:'Links',
    5:'Porcentagem',
},inplace=True)
dfConsulta = dfConsulta.astype({
    'Nome_Fundo': str,
    'Valor_Fundo': np.float64,
    'Valor_dividendos': np.float64,
    'Porcentagem': np.float64,
    'Data': 'datetime64[ns]',
    'Links': str,
})

